# Load and test trained model

## Load libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
from enduro_lstm import *

In [8]:
device = conf_cuda(True)

GPU is available


## Set configurations

In [2]:
data_path = r"../1-generate/data/"
match = 35
use_cuda = False
zigzag = True

## Load trained model

In [9]:
newpath = 'models/zigzag_m35to35_f1to120_epoch5600_H200/' + 'zigzag_m35to35_f1to120_epoch5600_H200'

In [11]:
model = Model(device=device, input_size=20400, output_size=2, hidden_dim=200, n_layers=1)
model.load_state_dict(torch.load(newpath))
model.eval()

Model(
  (lstm): LSTM(20400, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

## Prepare cell with trained model 

In [13]:
lstmcell = nn.LSTMCell(20400, 200)
linear = nn.Linear(200, 2)
sigmoid = nn.Sigmoid()

In [14]:
lstmcell.weight_ih = model.lstm.weight_ih_l0
lstmcell.weight_hh = model.lstm.weight_hh_l0
lstmcell.bias_hh = model.lstm.bias_hh_l0
lstmcell.bias_ih = model.lstm.bias_ih_l0

In [15]:
linear.weight = model.fc.weight
linear.bias = model.fc.bias

In [16]:
hx = torch.zeros(1, 200)
cx = torch.zeros(1, 200)

In [12]:
print(model.lstm.weight_ih_l0.shape)
print(model.lstm.weight_hh_l0.shape)
print(model.lstm.bias_ih_l0.shape)
print(model.lstm.bias_hh_l0.shape)
print(model.fc.weight.shape)
print(model.fc.bias.shape)

torch.Size([800, 20400])
torch.Size([800, 200])
torch.Size([800])
torch.Size([800])
torch.Size([2, 200])
torch.Size([2])


## Testing outputs of model

In [3]:
ACTIONS_LIST = get_actions_list(zigzag=zigzag)

In [4]:
num_of_frames, frames, actions, rewards, lifes = load_npz(data_path, match)

Successfully loaded NPZ.


In [5]:
X_train = frames.reshape(1, len(frames) ,-1)/255

In [6]:
action_one_hot = [prepare_action_data(i, ACTIONS_LIST) for i in actions]
Y_train = np.array(action_one_hot)
Y_train = Y_train.reshape(1, len(Y_train), -1)

In [7]:
X_train = torch.tensor(X_train).float()
Y_train = torch.tensor(Y_train).float()

In [17]:
output = []
for i in range(120):
    step_input = X_train[0][i]
    step_input = step_input.reshape(1, -1)
    hx, cx = lstmcell(step_input, (hx, cx))
    out = linear(hx)
    out = sigmoid(out)
    output.append(out)

In [18]:
for i in range(120):
    print(str(i) + ' - ' + str(output[i]))

0 - tensor([[0.9968, 0.0037]], grad_fn=<SigmoidBackward>)
1 - tensor([[0.9990, 0.0013]], grad_fn=<SigmoidBackward>)
2 - tensor([[9.9932e-01, 8.7661e-04]], grad_fn=<SigmoidBackward>)
3 - tensor([[9.9931e-01, 8.7995e-04]], grad_fn=<SigmoidBackward>)
4 - tensor([[9.9939e-01, 7.8353e-04]], grad_fn=<SigmoidBackward>)
5 - tensor([[9.9940e-01, 7.7714e-04]], grad_fn=<SigmoidBackward>)
6 - tensor([[9.9937e-01, 8.1386e-04]], grad_fn=<SigmoidBackward>)
7 - tensor([[9.9940e-01, 7.7012e-04]], grad_fn=<SigmoidBackward>)
8 - tensor([[9.9946e-01, 6.9736e-04]], grad_fn=<SigmoidBackward>)
9 - tensor([[9.9946e-01, 7.0052e-04]], grad_fn=<SigmoidBackward>)
10 - tensor([[9.9942e-01, 7.5344e-04]], grad_fn=<SigmoidBackward>)
11 - tensor([[9.9941e-01, 7.5958e-04]], grad_fn=<SigmoidBackward>)
12 - tensor([[9.9942e-01, 7.4992e-04]], grad_fn=<SigmoidBackward>)
13 - tensor([[9.9943e-01, 7.3849e-04]], grad_fn=<SigmoidBackward>)
14 - tensor([[9.9937e-01, 8.1102e-04]], grad_fn=<SigmoidBackward>)
15 - tensor([[9.9944e